In [1]:
#Kiểm tra thông tin GPU
!nvidia-smi

Fri Dec 27 10:35:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
#Tải thư viện YOLOv10 từ GitHub
!pip install -q git+https://github.com/THU-MIG/yolov10.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
#Cấp quyền cho Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
#Tải tập tin pretrain
%cd '/content/drive/MyDrive'
!mkdir YOLOv10
%cd '/content/drive/MyDrive/YOLOv10'
!mkdir pretrain
%cd pretrain
!wget https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt

/content/drive/MyDrive
mkdir: cannot create directory ‘YOLOv10’: File exists
/content/drive/MyDrive/YOLOv10
mkdir: cannot create directory ‘pretrain’: File exists
/content/drive/MyDrive/YOLOv10/pretrain
--2024-12-27 11:11:10--  https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/THU-MIG/yolov10/releases/download/v1.0/yolov10n.pt [following]
--2024-12-27 11:11:10--  https://github.com/THU-MIG/yolov10/releases/download/v1.0/yolov10n.pt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/804788522/de01476f-8157-4901-921f-e0c6cb3848cf?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241227%2Fus-east-1%

In [7]:
#Tải và giải nén thư mục dữ liệu huấn luyện, file nén phải được tải lên đúng thư mục YOLOv10.
#Thư mục dữ liệu phải đúng định dạng theo YOLOv7 trở lên. Nếu chưa đúng, vui lòng tinh chỉnh lại
%cd '/content/drive/MyDrive/YOLOv10'
!unzip train_data.zip

/content/drive/MyDrive/YOLOv10
Archive:  train_data.zip
   creating: train_data/
  inflating: train_data/data.yaml    
  inflating: train_data/README.dataset.txt  
  inflating: train_data/README.roboflow.txt  
   creating: train_data/test/
   creating: train_data/test/images/
  inflating: train_data/test/images/Chair--269-_jpg.rf.87aa2b7101b1fd216cd9762a0283f4ad.jpg  
  inflating: train_data/test/images/Chair--270-_jpg.rf.77fe75ee33dd358303bd3406412ff8dd.jpg  
  inflating: train_data/test/images/Chairs--1-_jpg.rf.7104107727daae1f8c000a66cf0dd7b1.jpg  
  inflating: train_data/test/images/Chairs--10-_jpg.rf.cb94c97083903d1228907f324283816a.jpg  
  inflating: train_data/test/images/Chairs--11-_jpg.rf.731df0193231d047877e2b6e7b1cae30.jpg  
  inflating: train_data/test/images/Chairs--15-_jpg.rf.9be74e388453b5ecc3adb2b5a8fd5986.jpg  
  inflating: train_data/test/images/Chairs--16-_jpg.rf.ddca24c24866d8fe5ce85ff1693d476d.jpg  
  inflating: train_data/test/images/Chairs--17-_jpg.rf.55c5e0f6af8

In [14]:
import os
#Chỉ cần import trực tiếp thư viện YOLOv10, không cần cài đặt thêm thư viện khác
from ultralytics import YOLOv10
#Tắt chế độ WanDB (không ảnh hưởng đến model) để tránh việc bị yêu cầu nhập API.
os.environ["WANDB_MODE"] = "disabled"
%cd '/content/drive/MyDrive/YOLOv10'
#Huấn luyện model
!yolo task=detect mode=train epochs=50 batch=32 plots=True model='./pretrain/yolov10n.pt' data='/content/drive/MyDrive/YOLOv10/train_data/data.yaml'

/content/drive/MyDrive/YOLOv10
/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map

In [6]:
#Test thử kết quả
from ultralytics import YOLOv10
%cd '/content/drive/MyDrive/YOLOv10'
model_path = '/content/drive/MyDrive/YOLOv10/runs/detect/train/weights/best.pt'
model = YOLOv10(model_path)
#Kéo ảnh test_image trực tiếp vào content để test nhanh, hoặc tìm ảnh khác, có thể dùng URL ảnh để test kết quả
results = model(source='/content/test_image.jpg', conf=0.25,save=True)

/content/drive/MyDrive/YOLOv10

image 1/1 /content/test_image.jpg: 448x640 1 Chair, 1 Sofa, 59.0ms
Speed: 15.0ms preprocess, 59.0ms inference, 445.8ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict4
